In [6]:
import cv2
import numpy as np
from torch import device
import sys
sys.path.append('./yolov7/')

from yolov7.predict import load_model, preprocessing, \
    postprocessing, load_config

In [9]:
#Load config from yaml
CONFIG_PATH = './yolov7/predict_config.yaml'
config_dict = load_config(CONFIG_PATH)

#Read img0 to BGR
img0 = cv2.imread('./yolov7/inference/images/image1.jpg')
original_shape = img0.shape 

In [10]:
#Load model, edit yolov7.pt
model, stride, device = load_model(config_dict, '../yolov7.pt')


YOLOR 🚀 a0bbdb1 torch 1.12.1+cu102 CPU



IndexError: list index out of range

In [11]:

#Resize, pad
img = preprocessing(config_dict, img0, stride, device) 
inference_shape = img.shape

#Inference
pred = model(img, augment=config_dict['augment'])[0] #Shape (1, num_preds, 85)

#nms and scale coordinates
pred = postprocessing(config_dict, pred, original_shape, inference_shape)


NameError: name 'stride' is not defined